# Lab 14 — Attention & Transformer

> **강의 시간:** 약 2시간  
> **주제:** Attention 메커니즘, Positional Encoding, Transformer Encoder, 감성 분류

---

## 학습 목표

| # | 목표 | 예상 시간 |
|---|---|---|
| 1 | Scaled Dot-Product Attention 수동 구현 및 히트맵 시각화 | 30분 |
| 2 | Positional Encoding 구현 + Multi-Head Attention 패턴 비교 | 30분 |
| 3 | Transformer Encoder 조립 + 합성 감성 데이터 분류 학습 | 30분 |
| 4 | Exercise | 30분 |

---

**데이터셋:** numpy 합성 감성 데이터 (인터넷 불필요)  
- `vocab_size=100` (0=PAD, 1=CLS, 2\~49=긍정 단어, 50\~99=부정 단어)  
- `seq_len=20` (CLS 토큰 + 내용 토큰 19개), `n_samples=2000` (train 1600 / test 400)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

_fp = '/System/Library/Fonts/AppleGothic.ttf'
fm.fontManager.addfont(_fp)
plt.rcParams['font.family'] = fm.FontProperties(fname=_fp).get_name()
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

torch.manual_seed(42)
np.random.seed(42)

if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f'PyTorch version: {torch.__version__}')
print(f'Using device   : {device}')


In [ ]:
# ── 합성 감성 데이터 생성 ─────────────────────────────────────────────────────
# vocab: 0=PAD, 1=CLS, 2~49=긍정 단어(48개), 50~99=부정 단어(50개)

VOCAB_SIZE = 100
SEQ_LEN    = 20    # CLS(1) + 내용 토큰 19개
N_SAMPLES  = 2000  # train 1600 / test 400
POS_RANGE  = (2,  50)   # 긍정 토큰 범위 [2, 49]
NEG_RANGE  = (50, 100)  # 부정 토큰 범위 [50, 99]


def generate_sentiment_data(n_samples=N_SAMPLES, seq_len=SEQ_LEN, seed=42):
    """
    긍정(label=1) / 부정(label=0) 합성 감성 시퀀스 생성
    - 각 시퀀스는 CLS(=1) 토큰으로 시작
    - 긍정: 내용 토큰의 약 60%가 긍정 영역(2~49)
    - 부정: 내용 토큰의 약 60%가 부정 영역(50~99)
    """
    rng = np.random.RandomState(seed)
    tokens_list, labels = [], []
    content_len = seq_len - 1            # CLS 제외 내용 길이
    n_signal    = int(content_len * 0.6) # 60% 신호 토큰
    n_noise     = content_len - n_signal # 40% 노이즈 토큰
    for _ in range(n_samples):
        label = rng.randint(0, 2)
        if label == 1:   # 긍정
            signal = rng.randint(*POS_RANGE, size=n_signal)
            noise  = rng.randint(*NEG_RANGE, size=n_noise)
        else:            # 부정
            signal = rng.randint(*NEG_RANGE, size=n_signal)
            noise  = rng.randint(*POS_RANGE, size=n_noise)
        content = np.concatenate([signal, noise])
        rng.shuffle(content)
        seq = np.concatenate([[1], content])  # CLS 토큰 맨 앞
        tokens_list.append(seq)
        labels.append(label)
    X = torch.LongTensor(np.array(tokens_list))  # (N, seq_len)
    y = torch.LongTensor(np.array(labels))        # (N,)
    return X, y


def split_data(X, y, train_ratio=0.8):
    n = int(len(X) * train_ratio)
    return X[:n], y[:n], X[n:], y[n:]


X_all, y_all   = generate_sentiment_data()
X_tr, y_tr, X_te, y_te = split_data(X_all, y_all)
train_ds  = TensorDataset(X_tr, y_tr)
train_ldr = DataLoader(train_ds, batch_size=64, shuffle=True)

print('=== 합성 감성 데이터셋 ===')
print(f'  vocab_size : {VOCAB_SIZE}  (0=PAD, 1=CLS, 2~49=긍정, 50~99=부정)')
print(f'  seq_len    : {SEQ_LEN}   (CLS + 내용 {SEQ_LEN-1}토큰)')
print(f'  총 샘플    : {len(X_all)}  (Train {len(X_tr)} / Test {len(X_te)})')
print(f'  긍정 비율  : {y_all.float().mean():.2f}')
print('\n샘플 시퀀스 (처음 3개):')
for i in range(3):
    lbl = '긍정' if y_all[i].item() == 1 else '부정'
    print(f'  [{lbl}] {X_all[i].tolist()}')

# ── 시각화 ──────────────────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(13, 4))
for ax_idx, label_val in enumerate([1, 0]):
    idxs = (y_all == label_val).nonzero(as_tuple=True)[0][:5]
    lbl_str = '긍정(label=1)' if label_val == 1 else '부정(label=0)'
    pos_c, neg_c = [], []
    for idx in idxs:
        seq = X_all[idx, 1:].numpy()  # CLS 제외
        pos_c.append(int((seq < 50).sum()))
        neg_c.append(int((seq >= 50).sum()))
    x = np.arange(5)
    axes[ax_idx].bar(x - 0.2, pos_c, width=0.4, label='긍정 토큰(2~49)', color='seagreen', alpha=0.8)
    axes[ax_idx].bar(x + 0.2, neg_c, width=0.4, label='부정 토큰(50~99)', color='tomato',  alpha=0.8)
    axes[ax_idx].set_title(f'{lbl_str} 샘플 5개의 토큰 분포')
    axes[ax_idx].set_xlabel('샘플 인덱스'); axes[ax_idx].set_ylabel('토큰 수')
    axes[ax_idx].set_xticks(x); axes[ax_idx].legend(fontsize=9)
plt.suptitle('합성 감성 데이터 — 토큰 분포', fontsize=12)
plt.tight_layout(); plt.show()


---
## Part 1. Scaled Dot-Product Attention

### 1-1. 핵심 수식

$$\\text{Attention}(Q, K, V) = \\text{softmax}\\!\\left(\\frac{QK^\\top}{\\sqrt{d_k}}\\right)V$$

| 기호 | 크기 | 설명 |
|---|---|---|
| $Q$ (Query) | $(\\text{seq\\_len},\\, d_k)$ | "무엇을 찾는가" |
| $K$ (Key)   | $(\\text{seq\\_len},\\, d_k)$ | "어디에 정보가 있는가" |
| $V$ (Value) | $(\\text{seq\\_len},\\, d_v)$ | "실제 정보" |
| $\\sqrt{d_k}$ 나눗셈 | 스칼라 | 내적값이 커지는 문제 방지 (softmax 포화 방지) |

### 1-2. 왜 √d_k로 나누는가?
- Q, K가 평균 0, 분산 1이면 $QK^\\top$의 분산 = $d_k$
- √d_k로 나누면 분산 = 1 → softmax 입력이 너무 크면 그래디언트가 사라짐

### 1-3. Self-Attention
- Q = K = V = **같은 시퀀스** → 자기 자신 내부의 모든 위치 관계를 동시에 계산
- RNN과 달리 모든 위치가 **직접** 연결됨 → 장기 의존성 문제 해결


In [ ]:
# ── Scaled Dot-Product Attention 수동 구현 ─────────────────────────────────

def sdpa_numpy(Q, K, V, mask=None):
    """
    Scaled Dot-Product Attention (numpy)
    Q, K: (seq_len, d_k)   V: (seq_len, d_v)
    mask: (seq_len, seq_len) bool — True인 위치를 -inf로 마스킹
    """
    d_k    = Q.shape[-1]
    scores = Q @ K.T / np.sqrt(d_k)              # (seq_len, seq_len)
    if mask is not None:
        scores[mask] = -np.inf
    # 수치 안정화 softmax
    scores  = scores - scores.max(axis=-1, keepdims=True)
    exp_s   = np.exp(scores)
    weights = exp_s / exp_s.sum(axis=-1, keepdims=True)  # (seq_len, seq_len)
    output  = weights @ V                                  # (seq_len, d_v)
    return output, weights


def sdpa_torch(Q, K, V, mask=None):
    """Scaled Dot-Product Attention (PyTorch 검증용)"""
    d_k    = Q.shape[-1]
    scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask, float('-inf'))
    weights = F.softmax(scores, dim=-1)
    return torch.matmul(weights, V), weights


# ── 데모: 랜덤 Q, K, V ───────────────────────────────────────────────────────
np.random.seed(0)
seq_demo, d_k_demo, d_v_demo = 6, 8, 8

Q_np = np.random.randn(seq_demo, d_k_demo).astype(np.float32)
K_np = np.random.randn(seq_demo, d_k_demo).astype(np.float32)
V_np = np.random.randn(seq_demo, d_v_demo).astype(np.float32)

out_np, w_np = sdpa_numpy(Q_np, K_np, V_np)

Q_pt = torch.FloatTensor(Q_np)
K_pt = torch.FloatTensor(K_np)
V_pt = torch.FloatTensor(V_np)
out_pt, w_pt = sdpa_torch(Q_pt, K_pt, V_pt)

max_out_diff = np.max(np.abs(out_np - out_pt.numpy()))
max_w_diff   = np.max(np.abs(w_np  - w_pt.numpy()))
print(f'numpy vs PyTorch 최대 오차 — output: {max_out_diff:.2e},  weights: {max_w_diff:.2e}')
print('(≈0 이면 두 구현이 동일)\n')

print(f'어텐션 가중치 행별 합 (softmax → 항상 1.0):')
print(f'  numpy  : {w_np.sum(axis=-1).round(4)}')
print(f'  PyTorch: {w_pt.sum(dim=-1).numpy().round(4)}\n')

print(f'텐서 크기 요약:')
print(f'  Q, K : {Q_np.shape}  →  scores : ({seq_demo}, {seq_demo})')
print(f'  weights (softmax) : ({seq_demo}, {seq_demo})')
print(f'  V : {V_np.shape}  →  output : ({seq_demo}, {d_v_demo})')


In [ ]:
# ── Self-Attention 히트맵 시각화 ────────────────────────────────────────────

np.random.seed(7)
seq_vis = SEQ_LEN
d_k_vis = 16

Q_vis = np.random.randn(seq_vis, d_k_vis).astype(np.float32)
K_vis = np.random.randn(seq_vis, d_k_vis).astype(np.float32)
V_vis = np.random.randn(seq_vis, d_k_vis).astype(np.float32)

# 마스킹 없음
_, w_no_mask = sdpa_numpy(Q_vis, K_vis, V_vis)

# 마지막 5토큰을 패딩으로 가정 → Key 열 마스킹
mask_vis = np.zeros((seq_vis, seq_vis), dtype=bool)
mask_vis[:, -5:] = True
_, w_masked = sdpa_numpy(Q_vis, K_vis, V_vis, mask=mask_vis)

tick_lbl = ['CLS'] + [f't{i}' for i in range(1, seq_vis)]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

im0 = axes[0].imshow(w_no_mask, cmap='Blues', vmin=0, vmax=w_no_mask.max())
axes[0].set_title('Self-Attention 가중치 (마스킹 없음)')
axes[0].set_xlabel('Key 위치 (어디에 집중?)')
axes[0].set_ylabel('Query 위치 (어느 토큰이?)')
axes[0].set_xticks(range(seq_vis)); axes[0].set_xticklabels(tick_lbl, fontsize=7, rotation=45)
axes[0].set_yticks(range(seq_vis)); axes[0].set_yticklabels(tick_lbl, fontsize=7)
plt.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

im1 = axes[1].imshow(w_masked, cmap='Blues', vmin=0, vmax=w_masked.max())
axes[1].set_title('Self-Attention 가중치 (마지막 5토큰 마스킹)')
axes[1].set_xlabel('Key 위치'); axes[1].set_ylabel('Query 위치')
axes[1].set_xticks(range(seq_vis)); axes[1].set_xticklabels(tick_lbl, fontsize=7, rotation=45)
axes[1].set_yticks(range(seq_vis)); axes[1].set_yticklabels(tick_lbl, fontsize=7)
# 마스킹 영역 표시 (빨간 세로선)
for col in range(seq_vis - 5, seq_vis):
    axes[1].axvline(x=col - 0.5, color='red', lw=0.7, alpha=0.7)
plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

plt.suptitle('Scaled Dot-Product Self-Attention 가중치 히트맵\n'
             '행 = Query 위치, 열 = Key 위치, 밝을수록 높은 가중치', fontsize=11)
plt.tight_layout(); plt.show()

print('📌 핵심 관찰:')
print('  - 각 Query는 모든 Key 위치에 어텐션 가중치를 부여합니다.')
print('  - 마스킹된 위치(빨간 선 오른쪽)는 어텐션에서 완전히 제외됩니다.')
print(f'  - 가중치 행별 합 = 1: {w_masked.sum(axis=-1).round(3)}')


---
## Part 2. Positional Encoding + Multi-Head Attention

### 2-1. Positional Encoding

Transformer는 **순서 정보가 없다** → sin/cos 함수로 위치 정보를 임베딩에 주입:

$$PE_{(pos,\, 2i)}   = \\sin\\!\\left(\\frac{pos}{10000^{2i/d_{\\text{model}}}}\\right)$$
$$PE_{(pos,\, 2i+1)} = \\cos\\!\\left(\\frac{pos}{10000^{2i/d_{\\text{model}}}}\\right)$$

- **낮은 차원** (i 작음): 빠르게 변하는 패턴 → 가까운 위치 구분  
- **높은 차원** (i 큼): 느리게 변하는 패턴 → 먼 위치 구분  
- `register_buffer` → 학습 파라미터 없음, 고정값

### 2-2. Multi-Head Attention

단일 어텐션은 1가지 관계만 포착 → **h개 헤드를 병렬로** 실행해 다양한 관계 동시 포착:

$$\\text{MultiHead}(Q,K,V) = \\text{Concat}(\\text{head}_1,\\ldots,\\text{head}_h)W^O$$
$$\\text{head}_i = \\text{Attention}(QW_i^Q,\\, KW_i^K,\\, VW_i^V)$$

| 비교 | Single-Head | Multi-Head (h=4) |
|---|---|---|
| Q/K/V 차원 | $d_k = d_{\\text{model}}$ | $d_k = d_{\\text{model}} / h$ |
| 어텐션 계산 | 1회 | h회 병렬 |
| 총 파라미터 | $4d^2$ | $4d^2$ (동일!) |


In [ ]:
# ── Positional Encoding 구현 및 시각화 ──────────────────────────────────────

class PositionalEncoding(nn.Module):
    """
    Sinusoidal Positional Encoding (Vaswani et al. 2017)
    PE(pos, 2i)   = sin(pos / 10000^(2i/d_model))
    PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
    """
    def __init__(self, d_model, max_len=512, dropout=0.0):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe       = torch.zeros(max_len, d_model)                     # (max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()     # (max_len, 1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
        )                                                             # (d_model/2,)
        pe[:, 0::2] = torch.sin(position * div_term)                 # 짝수 차원
        pe[:, 1::2] = torch.cos(position * div_term)                 # 홀수 차원
        pe = pe.unsqueeze(0)                                         # (1, max_len, d_model)
        self.register_buffer('pe', pe)                               # 학습 파라미터 아님

    def forward(self, x):
        """x: (B, seq_len, d_model)"""
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)


# ── 시각화 ──────────────────────────────────────────────────────────────────
d_model_vis = 64
max_len_vis = 50
pe_module   = PositionalEncoding(d_model_vis, max_len=max_len_vis)
pe_matrix   = pe_module.pe[0].numpy()    # (max_len, d_model)

fig, axes = plt.subplots(1, 2, figsize=(14, 4.5))

im = axes[0].imshow(pe_matrix, cmap='RdBu', vmin=-1, vmax=1, aspect='auto')
axes[0].set_title(f'PE 행렬 ({max_len_vis} 위치 × {d_model_vis} 차원)')
axes[0].set_xlabel('차원 (d_model)'); axes[0].set_ylabel('위치 (pos)')
plt.colorbar(im, ax=axes[0], fraction=0.046, pad=0.04)

pos_range = np.arange(max_len_vis)
for dim_idx, color in zip([0, 2, 4, 10, 20],
                          ['steelblue','tomato','seagreen','purple','orange']):
    axes[1].plot(pos_range, pe_matrix[:, dim_idx], color=color, lw=2, label=f'dim={dim_idx}')
axes[1].set_title('차원별 PE 값 (위치에 따른 sin/cos 주기)')
axes[1].set_xlabel('위치 (pos)'); axes[1].set_ylabel('PE 값')
axes[1].set_ylim(-1.1, 1.1); axes[1].axhline(y=0, color='gray', ls='--', lw=0.8)
axes[1].legend(fontsize=9)

plt.suptitle('Sinusoidal Positional Encoding 시각화\n'
             '차원 번호가 클수록 더 낮은 주파수 (천천히 변하는 패턴)', fontsize=11)
plt.tight_layout(); plt.show()

print('📌 PositionalEncoding 특성:')
print('  - 각 위치마다 고유한 d_model차원 벡터 부여')
print('  - 낮은 차원(0, 2): 빠르게 변하는 패턴 → 가까운 위치 구분')
print('  - 높은 차원(20):   느리게 변하는 패턴 → 먼 위치 구분')
print('  - register_buffer: 학습하지 않는 고정 파라미터')


In [ ]:
# ── Multi-Head Attention 헤드별 어텐션 패턴 시각화 ─────────────────────────

nhead_demo  = 4
d_model_mha = 64

torch.manual_seed(42)
mha_layer = nn.MultiheadAttention(
    embed_dim=d_model_mha, num_heads=nhead_demo, dropout=0.0, batch_first=True
)
emb_demo = nn.Embedding(VOCAB_SIZE, d_model_mha)

# 데이터셋에서 샘플 1개 추출 → 랜덤 임베딩으로 MHA 시연
sample_tok = X_tr[0:1]   # (1, seq_len)
with torch.no_grad():
    x_mha = emb_demo(sample_tok)    # (1, seq_len, d_model)
    # average_attn_weights=False → 각 헤드별 가중치 반환 (PyTorch >= 1.13)
    _, attn_w = mha_layer(
        x_mha, x_mha, x_mha,
        need_weights=True, average_attn_weights=False
    )    # attn_w: (1, nhead, seq_len, seq_len)

attn_np = attn_w[0].numpy()    # (nhead, seq_len, seq_len)
tick_lbl = ['CLS'] + [f't{i}' for i in range(1, SEQ_LEN)]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
for h in range(nhead_demo):
    ax = axes[h // 2][h % 2]
    im = ax.imshow(attn_np[h], cmap='Blues', vmin=0, vmax=attn_np[h].max())
    ax.set_title(f'Head {h+1} 어텐션 가중치', fontsize=10)
    ax.set_xlabel('Key 위치'); ax.set_ylabel('Query 위치')
    ax.set_xticks(range(SEQ_LEN)); ax.set_xticklabels(tick_lbl, fontsize=6, rotation=45)
    ax.set_yticks(range(SEQ_LEN)); ax.set_yticklabels(tick_lbl, fontsize=6)
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

plt.suptitle(f'Multi-Head Attention ({nhead_demo} heads) — 헤드별 어텐션 패턴\n'
             '각 헤드는 서로 다른 위치 관계에 집중합니다 (초기화 직후, 학습 전)', fontsize=11)
plt.tight_layout(); plt.show()

print(f'📌 Multi-Head Attention 핵심:')
print(f'  embed_dim={d_model_mha}, num_heads={nhead_demo}')
print(f'  각 헤드의 d_k = d_model / nhead = {d_model_mha // nhead_demo}')
print(f'  총 파라미터: {sum(p.numel() for p in mha_layer.parameters()):,}')
print(f'  → 헤드 수를 늘려도 총 파라미터는 Single-Head와 동일!')


---
## Part 3. Transformer Encoder + 감성 분류

### 3-1. Transformer Encoder Block 구조

```
입력 x
  │
  ├─→ [Multi-Head Self-Attention] ─→ Dropout ─→ + x ─→ LayerNorm ─→ z
  │                                                                    │
  └────────────────────────────────────────────────────────────────────┘
  z
  │
  ├─→ [FFN: Linear→ReLU→Dropout→Linear] ─→ + z ─→ LayerNorm ─→ output
  │
  └───────────────────────────────────────────────────────────────────
```

**FFN:** $\\text{FFN}(z) = \\text{ReLU}(zW_1 + b_1)W_2 + b_2$  
내부 차원 $d_{ff} = 4 \\times d_{\\text{model}}$ (원논문 관례)

| 구성요소 | 역할 |
|---|---|
| **Residual 연결** | 그래디언트 흐름 보호 (ResNet과 동일 원리) |
| **LayerNorm** | 각 레이어 출력을 정규화 → 학습 안정화 |
| **FFN** | 각 위치 독립적으로 비선형 변환 (위치 간 정보 공유 없음) |

### 3-2. CLS 토큰 분류 전략
- 시퀀스 맨 앞에 특수 `CLS(=1)` 토큰 추가
- Self-Attention을 통해 CLS가 전체 시퀀스 정보를 집약
- 최종 레이어의 CLS 위치 출력 → Linear → 클래스 예측
- BERT의 분류 방식과 동일한 원리


In [ ]:
# ── 모델 클래스 정의 ─────────────────────────────────────────────────────────

class TransformerEncoderBlock(nn.Module):
    """
    단일 Transformer Encoder 레이어
    x → MHA → Add & Norm → FFN → Add & Norm → output
    """
    def __init__(self, d_model, nhead, d_ff, dropout=0.1):
        super().__init__()
        self.attn  = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.ff    = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.drop  = nn.Dropout(dropout)
        self.last_attn_weights = None   # Exercise 2 용 저장 슬롯

    def forward(self, x, key_padding_mask=None):
        # 1) Multi-Head Self-Attention + Residual + LayerNorm
        attn_out, attn_w = self.attn(
            x, x, x,
            key_padding_mask=key_padding_mask,
            need_weights=True,
            average_attn_weights=False   # (B, nhead, seq, seq)
        )
        self.last_attn_weights = attn_w.detach().cpu()
        x = self.norm1(x + self.drop(attn_out))
        # 2) FFN + Residual + LayerNorm
        x = self.norm2(x + self.ff(x))
        return x


class TransformerSentimentClassifier(nn.Module):
    """
    Transformer Encoder 기반 감성 분류기
    Embedding → PositionalEncoding → N × EncoderBlock → CLS 풀링 → Linear
    """
    def __init__(self, vocab_size, d_model=64, nhead=4, d_ff=256,
                 n_layers=2, n_classes=2, dropout=0.1, use_pe=True):
        super().__init__()
        self.d_model   = d_model
        self.use_pe    = use_pe
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.pe        = PositionalEncoding(d_model, max_len=512, dropout=dropout)
        self.blocks    = nn.ModuleList([
            TransformerEncoderBlock(d_model, nhead, d_ff, dropout)
            for _ in range(n_layers)
        ])
        self.classifier = nn.Linear(d_model, n_classes)

    def forward(self, x, key_padding_mask=None):
        # 1) 토큰 임베딩 (√d_model 스케일링 — 원논문 관례)
        x = self.embedding(x) * math.sqrt(self.d_model)  # (B, seq_len, d_model)
        # 2) Positional Encoding
        if self.use_pe:
            x = self.pe(x)
        # 3) N개 EncoderBlock 통과
        for block in self.blocks:
            x = block(x, key_padding_mask=key_padding_mask)
        # 4) CLS 토큰 (위치 0) 출력으로 분류
        cls_out = x[:, 0, :]               # (B, d_model)
        return self.classifier(cls_out)    # (B, n_classes)


def count_params(model):
    return sum(p.numel() for p in model.parameters())


def train_transformer(model, train_loader, X_val, y_val,
                      n_epochs=30, lr=3e-4, verbose=True):
    """Transformer 감성 분류 학습 (CrossEntropyLoss + Adam)"""
    model     = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    X_v = X_val.to(device)
    y_v = y_val.to(device)
    train_losses, val_accs = [], []

    for epoch in range(n_epochs):
        model.train()
        ep_loss = 0.
        for Xb, yb in train_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            optimizer.zero_grad()
            logits = model(Xb)
            loss   = criterion(logits, yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            ep_loss += loss.item()
        train_losses.append(ep_loss / len(train_loader))

        model.eval()
        with torch.no_grad():
            val_logits = model(X_v)
            val_acc    = (val_logits.argmax(1) == y_v).float().mean().item()
        val_accs.append(val_acc)

        if verbose and (epoch + 1) % 5 == 0:
            print(f'Epoch {epoch+1:3d}/{n_epochs} | '
                  f'Train Loss: {train_losses[-1]:.4f} | Val Acc: {val_acc:.4f}')

    model.cpu()
    return train_losses, val_accs


# 파라미터 수 비교
print(f'{"n_layers":<10} {"파라미터 수":>14}')
print('-' * 26)
for nl in [1, 2, 4]:
    m = TransformerSentimentClassifier(VOCAB_SIZE, d_model=64, nhead=4, d_ff=256, n_layers=nl)
    print(f'{nl:<10} {count_params(m):>14,}')


In [ ]:
# ── 기본 모델 학습 ───────────────────────────────────────────────────────────
print('=== Transformer 감성 분류기 학습 (30 에포크) ===')
torch.manual_seed(42)
model_base = TransformerSentimentClassifier(
    vocab_size=VOCAB_SIZE, d_model=64, nhead=4, d_ff=256,
    n_layers=2, n_classes=2, dropout=0.1
)
print(f'파라미터 수: {count_params(model_base):,}\n')

base_losses, base_accs = train_transformer(
    model_base, train_ldr, X_te, y_te, n_epochs=30
)

model_base.eval()
with torch.no_grad():
    te_acc = (model_base(X_te).argmax(1) == y_te).float().mean().item()
print(f'\n최종 테스트 정확도: {te_acc:.4f}')

# 학습 곡선 시각화
ep = range(1, 31)
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

axes[0].plot(ep, base_losses, 'steelblue', lw=2)
axes[0].set_title('Train Loss (에포크별)')
axes[0].set_xlabel('에포크'); axes[0].set_ylabel('Cross-Entropy Loss')

axes[1].plot(ep, base_accs, 'seagreen', lw=2)
axes[1].axhline(y=0.5, color='gray', ls='--', lw=1, label='무작위 기준선 (0.5)')
axes[1].set_title('Val Accuracy (에포크별)')
axes[1].set_xlabel('에포크'); axes[1].set_ylabel('Accuracy')
axes[1].set_ylim(0, 1.05); axes[1].legend()

plt.suptitle(f'Transformer 감성 분류기 학습 결과\n'
             f'(d_model=64, nhead=4, n_layers=2, 최종 Val Acc: {base_accs[-1]:.4f})', fontsize=11)
plt.tight_layout(); plt.show()

print('\n📌 설명:')
print('  - CLS 토큰이 Self-Attention을 통해 전체 시퀀스 정보를 집약합니다.')
print('  - 합성 데이터에서 긍정/부정 토큰 비율 패턴을 Transformer가 학습합니다.')
print('  - 무작위 기준선(0.5) 대비 성능이 올라갈수록 학습이 잘 된 것입니다.')


---
## Exercise

### Exercise 1. Encoder 레이어 수 탐색

`n_layers ∈ {1, 2, 4}`로 `TransformerSentimentClassifier`를 학습하세요.

| n_layers | 비고 |
|---|---|
| 1 | 단일 Encoder 레이어 |
| 2 | Part 3와 동일 (기준선) |
| 4 | 깊은 Encoder |

**요구사항:**
- 동일 조건: `d_model=64, nhead=4, d_ff=256, dropout=0.1, n_epochs=30`
- 최종 Val Accuracy + 파라미터 수를 표로 출력
- 막대그래프 시각화 (x=n_layers, y=Val Acc)


In [ ]:
# Exercise 1: Encoder 레이어 수 탐색

layer_configs = [1, 2, 4]
ex1_results   = {}   # {n_layers: (val_acc, n_params)}

for n_layers in layer_configs:
    print(f'--- n_layers={n_layers} 학습 중 ---')
    torch.manual_seed(42)
    # Your code here: TransformerSentimentClassifier 생성 및 train_transformer 호출
    # m = TransformerSentimentClassifier(...)
    # losses, accs = train_transformer(m, train_ldr, X_te, y_te, n_epochs=30, verbose=False)
    # ex1_results[n_layers] = (accs[-1], count_params(m))
    pass

# Your code here: 결과 표 출력
# print(f'{"n_layers":>10} {"Val Acc":>10} {"파라미터":>12}')
# ...

# Your code here: 막대그래프 시각화 (x=n_layers, y=Val Acc)


### Exercise 2. 학습된 어텐션 히트맵 시각화

학습된 `model_base`에서 테스트 샘플 1개를 선택하여 **각 레이어, 각 헤드**의 어텐션 가중치를 시각화하세요.

**힌트:**  
- `model_base.blocks[i].last_attn_weights`는 forward 호출 후 자동으로 저장됩니다.  
- shape: `(1, nhead, seq_len, seq_len)`

**요구사항:**
1. `model_base.eval()` + `torch.no_grad()`로 테스트 샘플 forward
2. `n_layers × nhead` 크기의 subplot grid로 히트맵 시각화
3. 관찰: 레이어가 깊어질수록 CLS 토큰(행/열 0번)의 어텐션 패턴이 어떻게 변하는가?


In [ ]:
# Exercise 2: 학습된 어텐션 히트맵 시각화

sample_idx = 0
sample_tok = X_te[sample_idx:sample_idx+1]   # (1, seq_len)
label_str  = '긍정' if y_te[sample_idx].item() == 1 else '부정'
print(f'선택된 샘플: {label_str} (token: {sample_tok[0].tolist()})')

# Your code here:
# 1) model_base.eval() + torch.no_grad()로 forward 호출
#    → 각 block의 last_attn_weights가 자동 저장됨

# 2) n_layers × nhead 히트맵 그리기
#    힌트: model_base.blocks[i].last_attn_weights[0].numpy()  → (nhead, seq_len, seq_len)

# 3) 관찰 결과를 주석으로 작성:
# 예) 레이어 1의 CLS는 ... 에 집중, 레이어 2의 CLS는 ... 에 집중


### Exercise 3. (도전) nhead 탐색 + Positional Encoding 제거 실험

두 가지 실험을 수행하세요.

**실험 A — nhead 탐색:** `nhead ∈ {1, 2, 4, 8}` (`d_model=64`이므로 모두 나누어 떨어짐)  
- 각각 학습 후 최종 Val Accuracy 비교  
- 막대그래프 시각화

**실험 B — PE 제거 실험:** `TransformerSentimentClassifier(use_pe=False)`로 학습  
- PE 있음(기존 `base_accs`) vs PE 없음의 Val Accuracy 곡선 비교  
- 질문: 합성 감성 데이터에서 순서 정보(PE)가 얼마나 중요한가? 이유는?


In [ ]:
# Exercise 3 (도전): nhead 탐색 + PE 제거 실험

# ── 실험 A: nhead 탐색 ─────────────────────────────────────────────────────
nhead_configs = [1, 2, 4, 8]
ex3a_results  = {}   # {nhead: val_acc}

for nhead in nhead_configs:
    torch.manual_seed(42)
    # Your code here: TransformerSentimentClassifier(nhead=nhead, ...) 학습
    # m = TransformerSentimentClassifier(...)
    # _, accs = train_transformer(m, train_ldr, X_te, y_te, n_epochs=30, verbose=False)
    # ex3a_results[nhead] = accs[-1]
    pass

# Your code here: nhead별 Val Acc 막대그래프


# ── 실험 B: PE 제거 실험 ───────────────────────────────────────────────────
torch.manual_seed(42)
# Your code here: TransformerSentimentClassifier(use_pe=False) 학습
# model_no_pe = TransformerSentimentClassifier(..., use_pe=False)
# _, accs_nope = train_transformer(...)

# Your code here: PE 있음 vs 없음 비교 그래프 (base_accs vs accs_nope)

# 관찰 및 이유 작성 (주석):
# 합성 데이터에서 PE의 중요도:


---
## Summary

| 개념 | 핵심 내용 |
|---|---|
| **Query / Key / Value** | Q="찾고 싶은 것", K="어디 있나", V="실제 정보"; Attention = softmax(QKᵀ/√d_k)V |
| **√d_k 스케일링** | 내적 분산 = d_k → 나누면 분산 = 1 → softmax 포화 방지 |
| **Self-Attention** | Q=K=V=같은 시퀀스; 모든 위치 쌍을 동시에 계산 → RNN의 장기 의존성 문제 해결 |
| **Multi-Head Attention** | 헤드 h개가 d_model/h 차원에서 병렬 어텐션 → 다양한 관계 동시 포착, 파라미터는 동일 |
| **Positional Encoding** | sin/cos으로 위치 정보 주입; Transformer는 자체적으로 순서 모름 |
| **Encoder Block** | MHA → Add&Norm → FFN → Add&Norm; Residual + LayerNorm으로 학습 안정화 |
| **FFN** | Linear(d_model→d_ff) → ReLU → Linear(d_ff→d_model); d_ff = 4×d_model 관례 |
| **CLS 토큰** | 첫 위치에 고정 토큰 추가 → Self-Attention으로 전체 시퀀스 표현 집약 → 분류 |
| **O(n²) 복잡도** | 어텐션은 모든 위치 쌍 계산 → seq_len↑ 시 메모리/시간 급증; Flash Attention 등으로 개선 |
| **RNN vs Transformer** | RNN: 순차 처리, 장기 의존성 취약 / Transformer: 병렬 처리, 모든 위치 직접 연결 |

---

**다음 강의:** BERT, GPT — 사전학습(Pre-training) + 파인튜닝(Fine-tuning) 패러다임
